In [1]:
# !pip install torch==1.9
# !pip install torchtext==0.10.1
# Libraries
from konlpy.tag import Okt  
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter
from gensim.models import FastText
import torch
import ast

# Preliminaries
from torchtext.legacy.data import TabularDataset ,Field, LabelField ,BucketIterator, Example, Dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
# Models
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import ConcatDataset
from konlpy.tag import Okt  
# Training
import re
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import logging
from gensim.models import FastText
# Evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from tqdm import tqdm

In [2]:
train_data = pd.read_excel("./middle/after_tokenized_train.xlsx")[["tokenized_text","label"]]
test_data = pd.read_excel("./middle/after_tokenized_test.xlsx")[["tokenized_text","label"]]

In [3]:
okt=Okt()  
with open('./middle/all_speech_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

all_morphs = okt.morphs(text)
counted_morphs = Counter(all_morphs)

In [4]:
from tqdm import tqdm

clean_train_text = [ eval(text) for text in tqdm(train_data['tokenized_text'])]
clean_test_text = [ eval(text) for text in tqdm(test_data['tokenized_text'])]
clean_full_text = clean_train_text + clean_test_text

100%|██████████| 13491/13491 [00:00<00:00, 29979.57it/s]


In [5]:
fast_text_model = FastText(vector_size=100, window=3, min_count=1, sentences=clean_full_text, epochs=20)

In [6]:
fast_text_model.wv.most_similar("이명박")

[('이명숙', 0.9372692108154297),
 ('이명박정부', 0.9092457890510559),
 ('이명희', 0.8945683836936951),
 ('명박', 0.8714800477027893),
 ('이명', 0.8540940880775452),
 ('배근혜', 0.8535810112953186),
 ('닥근혜', 0.851435661315918),
 ('근혜', 0.840703547000885),
 ('문근혜', 0.840450644493103),
 ('최순실', 0.8337816596031189)]

In [7]:
tokens = list(counted_morphs.keys())
token_to_index = {token: i+1 for i, token in enumerate(tokens)}
index_to_token = {index: token for token, index in token_to_index.items()}
index_to_token[0] = '<unknown>'

In [8]:
def token_to_index_list(token_list):
    token_list = eval(token_list)
    ret = []
    for token in token_list:
        ret.append(token_to_index[token])
    return ret

In [9]:
train_data["indexed_token"] = train_data["tokenized_text"].map(token_to_index_list)
test_data["indexed_token"] = test_data["tokenized_text"].map(token_to_index_list)

In [10]:
indexed_tokens_list_test = [tokens for tokens in test_data['indexed_token']]
indexed_tokens_list_train = [tokens for tokens in train_data['indexed_token']]

In [11]:
train_data.head()

,tokenized_text,label,indexed_token
0,"['유소영', '비호감', '성형', '아줌마']",1,"[1, 2, 3, 4]"
1,"['나오지마라', '썅']",3,"[5, 6]"
2,"['식상하고', '지긋지긋했는데', '잘', '끝나네', '오예', '소리', '벗...",6,"[7, 8, 9, 10, 11, 12, 13, 14, 15]"
3,"['성희롱', '당할', '얼굴', '아닌데', 'ㅋㅋ']",5,"[16, 17, 18, 20, 21]"
4,"['끝', '까지', '해보자', '쪽바리', '원숭이', '자유', '한국', '...",0,"[22, 23, 24, 25, 26, 27, 28, 29, 30]"


In [12]:
import torch.nn.functional as F

def pad_sequences(sequences, max_len=None):
    if max_len is None:
        max_len = max([len(seq) for seq in sequences])
    padded_seqs = torch.zeros((len(sequences), max_len)).long()
    for i, seq in enumerate(sequences):
        end = min(len(seq), max_len)
        padded_seqs[i, :end] = torch.LongTensor(seq[:end])
    return padded_seqs

In [13]:
train_input_list = pad_sequences(indexed_tokens_list_train,121)
test_input_list = pad_sequences(indexed_tokens_list_test,121)
train_answer_list = [int(label) for label in train_data['label']]
train_answer_list = torch.tensor(train_answer_list)

In [14]:
print(train_input_list.shape)
print(test_input_list.shape)
print(len(train_answer_list))

torch.Size([65863, 121])
torch.Size([13491, 121])
65863


In [15]:
# tokens = list(counted_morphs.keys())
# token_to_index = {token: i+1 for i, token in enumerate(tokens)}
# index_to_token = {index: token for token, index in token_to_index.items()}
# index_to_token[0] = '<unkown>'

final_train_input = []

for tmp_input_list in train_input_list:
    ret = []
    for tmp in tmp_input_list:
        tmp_to_word = index_to_token[int(tmp)]
        if tmp_to_word == "<unknown>":
            arr = np.zeros(100)
        arr = fast_text_model.wv[tmp_to_word]
        arr = arr.tolist()
        ret.append(arr)
    final_train_input.append(ret)
    
final_train_input = np.array(final_train_input)

print(final_train_input.shape)

(65863, 121, 100)


In [16]:
final_test_input = []

for tmp_input_list in test_input_list:
    ret = []
    for tmp in tmp_input_list:
        tmp_to_word = index_to_token[int(tmp)]
        if tmp_to_word == "<unknown>":
            arr = np.zeros(100)
        arr = fast_text_model.wv[tmp_to_word]
        arr = arr.tolist()
        ret.append(arr)
    final_test_input.append(ret)
    
final_test_input = np.array(final_test_input)

print(final_test_input.shape)

(13491, 121, 100)


In [17]:
final_test_input = torch.Tensor(final_test_input)
final_train_input = torch.Tensor(final_train_input)

torch.save(final_train_input, './processed_data/fasttext_train_input.pt')
torch.save(final_test_input,  './processed_data/fast_text_test_input.pt')
torch.save(train_answer_list, './processed_data/train_label.pt')